"""
Created on Sun Sept  29 11:38:07 2024

@author: MIT Wind

Wake loss simulation, farm layout optimization, hydrogen exploration

Assumptions:
    No blockage effect considered. We are using Squared Sum Superposition for the wake models
    Any wake model considered uses the default parameters with which it was validated
"""

In [2]:
import numpy as np
import pandas as pd 
from Site import Kratos, VWT45, wt_x, wt_y
from py_wake.deficit_models.gaussian import BastankhahGaussian,BastankhahGaussianDeficit
from py_wake.superposition_models import SquaredSum
from py_wake.wind_farm_models import PropagateDownwind
from py_wake.turbulence_models import CrespoHernandez, STF2017TurbulenceModel
import os
from py_wake.utils.plotting import setup_plot
import matplotlib.pyplot as plt 
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from py_wake.flow_map import HorizontalGrid
from py_wake.flow_map import XYGrid
from py_wake import XZGrid
from py_wake.examples.data import example_data_path
import utm

from topfarm.cost_models.py_wake_wrapper import PyWakeAEPCostModelComponent
from topfarm.cost_models.cost_model_wrappers import CostModelComponent
from topfarm import TopFarmProblem
from topfarm.easy_drivers import EasyScipyOptimizeDriver
from topfarm.plotting import NoPlot, XYPlotComp
from topfarm.constraint_components.spacing import SpacingConstraint
from topfarm.constraint_components.boundary import CircleBoundaryConstraint, XYBoundaryConstraint
from topfarm.constraint_components.constraint_aggregation import ConstraintAggregation
from topfarm.constraint_components.constraint_aggregation import DistanceConstraintAggregation

# Plot wind rose from Global wind Atlas
# plt.figure()
# Kratos.plot_wd_distribution(n_wd=12)

In [ ]:
# Optimization around the island
n_wts = 1 # Number of turbines in the farm

def kratos_constraints(diam, boundary, n_wt=n_wts):
    """Constraints for Kratos

    n_wt : int, optional. Number of wind turbines in farm (default is 4)
    boundary: list of tuples representing vertices position

    Returns constr : list of topfarm constraints
        Currently only have Spacing constraint and boundary constraint 
    """
    spac_constr = SpacingConstraint(2 * diam) # Minimum constraints
    bound_constr = XYBoundaryConstraint(boundary, 'convex_hull')
    return [spac_constr, bound_constr]

def bound_vertices(corners):
    """ Define array of vertices for the farm
    Pass a list of latitude and longitude tuples
    """
    return np.array(corners)

In [ ]:
# Max output after optimization
site_corners = [" Pass in coordinates of corners as tuples"]
site = Kratos(wt_x, wt_y)
wfm = PropagateDownwind(site, VWT45, wake_deficitModel=BastankhahGaussianDeficit(use_effective_ws=False),
                                        superpositionModel=SquaredSum(), deflectionModel=None) # Wind farm model
objective = PyWakeAEPCostModelComponent(wfm, n_wts) # Wrapper for the objective function (AEP including wakes)

tf = TopFarmProblem(
        design_vars= {'x':wt_x, 'y': wt_y},
        driver=EasyScipyOptimizeDriver(),
        cost_comp= objective,
        constraints= kratos_constraints(90,bound_vertices(site_corners)),
        plot_comp=XYPlotComp(plot_initial=False),
        ) # TopFarm problem definition
    
cost, op_state, _= tf.optimize()

In [ ]:
Kratos.ds

Wake loss calculation for new layout

In [ ]:
# Layout optimization visualization
sim_res = wfm(wt_x, wt_y) # intial simulation result
wt_x_op, wt_y_op = op_state['x'], op_state['y'] # Optimized layout

# Remember to pass in the 1 hour average wind speed data that we downloaded from the CSV
sim_res_op = wfm(wt_x_op, wt_y_op) # Simulate result for optimize turbine placement

# Time series wind data 
frame_8760 = pd.read_csv('speed_data.csv')
dir_120 = np.array(frame_8760['Dir_120']) # Change
ws_120 = np.array(frame_8760['wind speed at 100m (m/s)'])
ti_120 = np.array(frame_8760['TI_120']) # Change

time_stamp = np.arange(len(dir_120))/7.5/24

# Optimum wind farm simulation with 8760 data
sim_res_time = wfm(wt_x_op, wt_y_op, # wind turbine positions
                            wd=dir_120, # Wind direction time series
                            ws=ws_120, # Wind speed time series
                            time=time_stamp, # time stamps
                      )

hourly_output = np.empty(0) # Hourly power output from the farm
for ix in range(len(ws_120)):
    hourly_output = np.append(hourly_output,sim_res.Power.sel(wt=0, wd=dir_120[ix], ws= ws_120[ix], method='nearest').data/1e6*4 )

d = np.load(example_data_path + "/time_series.npz")
n_days=366

# plot time series generation 
axes = plt.subplots(3,1, sharex=True, figsize=(16,10))[1]

for ax, (v,l) in zip(axes, [(dir_120, 'Wind direction [deg]'),(ws_120,'Wind speed [m/s]'),(hourly_output,'Hourly Output (MW)')]):
    ax.plot(time_stamp, v)
    ax.set_ylabel(l, fontsize= 18)
_ = ax.set_xlabel('Time [day]', fontsize= 18)

In [ ]:
# Wake loss from Nrel data
aep_with_wake_loss_time = sim_res_time.aep().sum().data
aep_witout_wake_loss_time = sim_res_time.aep(with_wake_loss=False).sum().data
print('total wake loss:',((aep_witout_wake_loss_time-aep_with_wake_loss_time) / aep_witout_wake_loss_time)*100, '%')
print('Capacity factor:', (aep_with_wake_loss_time/105.12)*100, '%')
aep_with_wake_loss_time

Plot wake map

In [ ]:
wdir = 300 # Wind direction to plot flow map
wsp = 8 # Wind speed to plot flow map

plt.figure()
flow_map = sim_res_op.flow_map(grid=None,
                            wd=wdir,
                            ws = wsp).plot_wake_map()
plt.scatter(wt_x, wt_y, label='Initial Position', color = 'black', marker = 'o', s= 15)
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title('Layout and wake for'+ f' {wdir} deg and {wsp} m/s')
plt.legend()